References: 
* Clean Messages Title and Replace Emoticon title are editted from following code : https://www.kaggle.com/egebasturk1/yemeksepeti-sentiment-analysis/comments#The-Lemmatizer 
* Merge CSV Title is editted from following page:
https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #connect your drive to notebook

Mounted at /content/drive


Frameworks and packages that are used in this Notebook

In [ ]:
pip install jpype1

     |████████████████████████████████| 460kB 7.9MB/s 


In [ ]:
import jpype
import nltk
from nltk.corpus import stopwords
from string import punctuation, digits
import itertools
from pathlib import Path
import re
import os
from typing import List
from jpype import JClass, JString, getDefaultJVMPath, shutdownJVM, startJVM, java
import pandas as pd
from collections import Counter
from pathlib import Path
import nltk
import re
from string import punctuation, digits
import pandas as pd
import numpy as np
import glob
nltk.download("all")

# Reading CSV - DataFrame

In [ ]:
#While reading the csv file into df you may need to determine delimeter different than comma
df = pd.read_csv("path_of_csv_file.csv", delimiter = "~", encoding='latin-1') 
## also some turkish letters in some datasets are changed into other letters
replaceDict={'Ý':'İ','Ð':'Ğ','Þ':'Ş','þ':'ş','ý':'ı','ð':'ğ'} 

for x in replaceDict:
    df=df.replace(x,replaceDict[x],regex=True)
df.head()

In [ ]:
df["raw_text_column"]=df["raw_text_column"].astype(str)

# Replace Emoticon - Remove Links&Digits&Punctuations

In [ ]:
# Especially the social media texts have several emoji, we should validate those emoji's meaning as text.
def replaceemoticon(word):
    
    pos = re.findall(r"(?::\) | :-\) | =\) | :D | :d | <3 | \(: | :\'\) | \^\^|;\) | \(-:)", word)
    neg = re.findall(r"(:-\( | :\( | ;\( | ;-\( | =\( | :/ | :\\ | -_- | \): | \)-:)", word)
  
    if pos:
        #word = ":)"
        word = word + " pozitif"

    elif neg:
        #word = ":("
        word = word + " negatif"

    return word

In [ ]:
df["preProcessing"] = df["raw_text_column"].map(replaceemoticon)

In [ ]:
def preProcessing(text):

  return resubsatır(df_remove_punc(df_remove_digits(unlem(df_remove_email_adressed(df_remove_com_tr(df_remove_link(text)))))))

def df_remove_link(text):
  return re.sub(r"((http\S+)|(www\S+))", " ", text)

def df_remove_com_tr(text):
    text = re.sub(r'\S*\.(com|tr)'," ",text)
    return text

def df_remove_email_adressed(text):
  return re.sub(r'([.\w]{3,}@[.\w]{5,})', ' ', text)

def df_remove_digits(text): #remove 0123456789
    text=text.strip()
    remove_digits = str.maketrans(' ', ' ', digits)
    return text.translate(remove_digits)

def unlem(text): # '!' punctuation may have some sentiment meaning, that's why we turn it as text.
    if "!" in text:
      return text+" ünlem"
    else:
      return text

def resubsatır(text): #remove \n 
    texts= re.sub("\n"," ",text)
    return texts

def df_remove_punc(text): #remove punctuation
    regex = re.compile('[%s]' % re.escape(punctuation))
    return regex.sub(' ', text)


In [ ]:
%%time
## Remove http, com, link, digits:
df["preProcessing"] = df["preProcessing"].map(preProcessing)

In [ ]:
def replace_a(text): #replace â to a 
    if "â" in text:
      return text+"a"
    else:
      return text

In [ ]:
df["preProcessing"] = df["preProcessing"].map(replace_a)

# Editting the dataset content

In [ ]:
data=df.copy()

In [ ]:
pd.set_option("display.max_rows", data.shape[0])
pd.set_option('display.max_colwidth', data.shape[0])

Lower the text letters

In [ ]:
def lowers(text):
    text=re.sub("I","ı",text)
    text=re.sub("İ","i",text)
    text = text.lower()
    return text

data["preProcessing"] = data["preProcessing"].map(lowers)

Deleting duplicates and NA

In [ ]:
#Drop all dupplicates in the TICKET_DESC column

data.drop_duplicates(subset=['preProcessing'], inplace = True)

In [ ]:
# Drop the NA data

data.dropna(subset = ["preProcessing"], inplace = True)

(if needed) Deleting some terms that you dont want to in your dataset

In [ ]:
# bilgi verildi geçen verileri çıkartalım 
data = data[~data.preProcessing.str.contains(" bilgi verildi")]

Elimating too short and too long text length 

In [ ]:
# her satırdaki mesajın kelime sayısını 'totalwords' sütununa yaz
data['totalwords'] = data['preProcessing'].str.split().str.len()

In [ ]:
# You can determine the word length according to mean and standard deviation of word length in dataset
data = data[data.totalwords > 6]
data = data[data.totalwords < 61]

# Word Tokenize 

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
data['preProcessing'] = data.apply(lambda row: word_tokenize(row['preProcessing']), axis=1)

In [ ]:
data.iloc[100:110]

# Tokenize to String

In [ ]:
def tokenize_to_str(text):
  text= re.sub(r"'","", text)
  text= re.sub(r",","", text)
  text= re.sub(r"\[","", text)
  text= re.sub(r"\]","", text)
  return text

In [ ]:
def tokenizeList_to_str(text):
  return " ".join(text)

In [ ]:
data['preProcessing'] = data['preProcessing'].map(tokenizeList_to_str)

# Deleting speacial characters 

Deleting speacial characters that different from alphabet 

In [ ]:
karakterler=[]
def find(text):
  for i in text:
    if(i not in ["q","x","w","a","b","c","ç","d","e","f","g","ğ","h","ı","i","j","k","l","m","n","o","ö","p","r","s","ş","t","u","ü","v","y","z"]):
      karakterler.append(i)
    else:
      continue
data["preProcessing"].map(find)

In [ ]:
silinecek_unique = set(karakterler)
print(silinecek_unique)

{'ß', 'ô', '¶', '\x91', '¿', 'ì', 'ø', 'ï', '¾', '¸', 'ä', '«', 'ê', 'ú', '£', 'á', 'û', '°', '×', 'à', '±', 'ó', 'é', 'å', '·', '÷', '´', 'í', '\x92', '¨', 'ë', 'â', '²', '\x7f', 'æ', 'ã', 'î', '½', 'ñ', '¼', '§', 'õ', 'ù', 'è', 'ò', ' ', '³', '»', '¹', '©'}


In [ ]:
silinecek_unique = set(karakterler)
for i in silinecek_unique:
  data["preProcessing"]=data["preProcessing"].str.replace(i," ")

# Deleting one character in text 

In [ ]:
#### We are deleting one character remained in the text not two character because in Turkish you have word examples like "et,su,ev"
def sil_1(text):
    text_list=text.split()
    for i in text_list:
        if len(i)==1:
            text_list.remove(i)   
    return " ".join(text_list)

In [ ]:
data["preProcessing"]=data["preProcessing"].map(sil_1)

# Deleting Stopwords

In [ ]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_word_list.remove('ama') ##you can edit stopwords list- we don't want to delete "ama" because it has meaningful negativity in Turkish.

In [ ]:
def stopwords_sil(text):
    text_list=text.split()
    silinmis_list = [text for text in text_list if text not in stop_word_list]
    silinmis_list=" ".join(silinmis_list)
    return silinmis_list

In [ ]:
data["preProcessing"]=data["preProcessing"].map(stopwords_sil)

# Deleting English words 

In [ ]:
pip install fasttext #for detecting language 

In [ ]:
import fasttext
PRETRAINED_MODEL_PATH = 'drive/MyDrive/..path../lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [ ]:
def ing(text):
    predictions = model.predict([text], k=2) #it will give you the most possible two languge label
    if predictions[0][0][0] =="__label__tr":
        y=predictions[1][0][0]
        if y > 0.92: #you can determine the turkish language score according to your dataset
            return "tr" #this will labelled the rows as "tr" or "NaN" according to their lang score

In [ ]:
data["lang"]=data["preProcessing"].map(ing)

In [ ]:
tr_olmayan_df=data[data["lang"]!="tr"] ## how many rows are not Turkish?
tr_olmayan_df.shape

(1484, 10)

In [ ]:
data_ingsiz=data.dropna(subset=['lang']) #deleting NaN valued,which in Lang column, rows  

In [ ]:
data_ingsiz.shape

(830240, 10)

In [ ]:
data_ingsiz.to_csv(r'drive/My Drive/..yourpath../PreProcessing_pt1.csv', index=False) 

# Sampling from big df

In [ ]:
dataP = pd.read_csv(r'drive/My Drive/..yourpath../PreProcessing_pt1.csv')

In [ ]:
dataP['COUNTY_column'].value_counts().to_frame() ##for example : let's do sampling on preprocessed text according to each counties in İstanbul 

In [ ]:
dataP['COUNTY_column'].value_counts().index.tolist()

In [ ]:
##İstanbul  counties' list
ist_ilce = ['ADALAR', 'ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER', 'BAKIRKÖY', 'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ', 'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE', 'ÇATALCA', 'ÇEKMEKÖY', 'ESENLER', 'ESENYURT', 'EYÜPSULTAN', 'FATİH', 'GAZİOSMANPAŞA', 'GÜNGÖREN', 'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK', 'SANCAKTEPE', 'SARIYER', 'ŞİŞLİ', 'ŞİLE', 'SİLİVRİ', 'SULTANBEYLİ', 'SULTANGAZİ', 'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU']

In [ ]:
len(ist_ilce)

39

In [ ]:
df2 = dataP[dataP['COUNTY_column'].isin(ist_ilce)]

In [ ]:
df2.shape

(827985, 10)

In [ ]:
df2['COUNTY_column'].head()

0      ZEYTİNBURNU
1          AVCILAR
2    GAZİOSMANPAŞA
3       ARNAVUTKÖY
4       ARNAVUTKÖY
Name: X_IBB_COUNTY, dtype: object

In [ ]:
A=df2['COUNTY_column'].value_counts() #check-> how many county is valid in your dataset
## edit ist_ilce list according to this

In [ ]:
def ayrıdf(i):
  filter= df2['COUNTY_column'].str.contains(i, na=False )
  df3=df2.loc[filter]
  aa=pd.DataFrame(df3)
  b= df2.shape[0]/ {determined combined samples row shape}
  count=int(int(A.loc[i])/b)
  sampled=aa["Text_ID"].sample(n= count) #unique ID 
  df_pre= df_i.loc[sampled]
  df_pre.reset_index(inplace=True)
  return  df_pre.to_csv(r'drive/My Drive/..path../{0}.csv'.format(i), index=False)

In [ ]:
for i in ist_ilce:
  ayrıdf(i)

# Merge the little sample df's into one df

In [ ]:
import os
import glob
import pandas as pd
os.chdir("drive/My Drive/..path../{foldername}") #give the folder path of the sample dataframes

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd
comb=pd.read_csv("drive/My Drive/ilce/combined_csv.csv") #write the path to save combined_csv

In [ ]:
comb.shape

In [ ]:
comb['COUNTY_column'].value_counts()

# Clean Messages

We are using Zemberek packages for lemmatization, Turkish morphology , normalizer

In [ ]:
import jpype #you may need to download java program if you are working on your local
from jpype import getDefaultJVMPath,JClass
jar = r"drive/MyDrive/..path../zemberek-full.jar"
from jpype import getDefaultJVMPath,JClass
if not jpype.isJVMStarted():
    jpype.startJVM(getDefaultJVMPath(), classpath=[jar])

In [ ]:
ZEMBEREK_PATH = r'drive/MyDrive/..path../zemberek-full.jar'
TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
TurkishSpellChecker = JClass('zemberek.normalization.TurkishSpellChecker')
TurkishSentenceNormalizer = JClass('zemberek.normalization.TurkishSentenceNormalizer')
Paths = JClass('java.nio.file.Paths')
lookupRoot = Paths.get("drive/MyDrive/NLP/Kutuphaneler, Kaynaklar/normalization")
lmPath = Paths.get("drive/MyDrive/..path../data/lm/lm.2gram.slm")
morphology = TurkishMorphology.createWithDefaults()
morph = TurkishMorphology.createWithDefaults()
spell = TurkishSpellChecker(morph)
#you need to download lookuproot, lmp.2gram, zemberek_full.jar files in your drive 

CPU times: user 7.52 s, sys: 355 ms, total: 7.87 s
Wall time: 4.12 s
Parser   : 912 ms


In [ ]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')
stop_word_list.remove('ama')
WORDS = dict()
spell_folder = Path(r"{write_path}") 
#download big2.txt folder from https://www.kaggle.com/egebasturk1/yemeksepeti-sentiment-analysis/comments?select=big2.txt

def words(text): return re.findall(r'\w+', text.lower())
with open(os.path.expanduser(Path(spell_folder/ "big2.txt")), "r", encoding = 'utf-8') as f:
    for line in f:
        splitted = line.split()
        WORDS[splitted[0]] = int(splitted[1])

In [ ]:
def reverse(s): 
    if len(s) == 0: 
        return s 
    else: 
        return reverse(s[1:]) + s[0] 

def checkOpennes(word):
    vowels=['a','e','i','ı','o','ö','u','ü']
    open_vowels=['e','i','ü','ö']
    close_vowels=['a','ı','o','u']
    for i in range(len(word)):
        if reverse(word)[i] in vowels:
            if reverse(word)[i] in open_vowels:
                return True
            else:
                return False
        else:
            continue
def PresentCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if reverse(word)[i] in ei:
                return 'ei'
            elif reverse(word)[i] in üö:
                return 'üö'
            elif reverse(word)[i] in aı:
                return 'aı'
            elif reverse(word)[i] in uo:
                return 'uo'
            else:
                continue
    
def StartCheck(word):
        ei=['e','i']
        aı=['a','ı']
        üö=['ü','ö']
        uo=['u','o']
        for i in range(len(word)):
            if (word)[i] in ei:
                return 'ei'
            elif (word)[i] in üö:
                return 'üö'
            elif (word)[i] in aı:
                return 'aı'
            elif (word)[i] in uo:
                return 'uo'
            else:
                continue
def replaceall(s, n,a):
    occurence = s.count(n)
    alt = []
    temp = s
    for i in range(occurence):
        temp2 = temp
        for j in range(i,occurence):
            temp2 = temp2.replace(n,a,1)
            alt.append(temp2)
        temp = temp.replace(n,"!",1)
    for i in range(len(alt)):
        alt[i] = alt[i].replace("!",n)

    return alt

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    if  word in WORDS.keys():
        number = WORDS[word]
    else:
        number = 1
    if number == 0:
        number = 1
    return number / N

def correction(word):
      return max(candi(word), key=P)

def candi(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcçdefgğhıijklmnoöprsştuüvyzw'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
 
    sp     = replaceall(word,'ı','i')
    sp2     = replaceall(word,'u','ü')
    sp3    = replaceall(word,'o','ö')
    sp4     = replaceall(word,'g','ğ')
    sp5     = replaceall(word,'c','ç')
    sp6     = replaceall(word,'s','ş')
    sp7     = replaceall(word,'i','ı')
    sp8     = replaceall(word,'ö','o')
    sp9     = replaceall(word,'ş','s')
    sp10     = replaceall(word,'ğ','g')
    sp11     = replaceall(word,'ç','c')
    sp12     = replaceall(word,'ü','u')
    specials=[]
    specials.extend(sp)
    specials.extend(sp2)
    specials.extend(sp3)
    specials.extend(sp4)
    specials.extend(sp5)
    specials.extend(sp6)
    specials.extend(sp7)
    specials.extend(sp8)
    specials.extend(sp9)
    specials.extend(sp10)
    specials.extend(sp11)
    specials.extend(sp12)
    return set(deletes+transposes+replaces+inserts+specials)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
def print_diff(word, s):
    if not word == s:
        print(word + " --> " + s)
counter = 0


def lemmatizer(word,texts):
        wordList=[]
        wordList = re.sub("[^\w]", " ",  texts).split()
       
        pos=wordList.index(word)
     
        sakin=''
        word=correction(word)
        
        if len(wordList)-pos>3 and pos>2:
            for i, kelime in enumerate(wordList[pos-3:pos+4]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos>5:
            for i, kelime in enumerate(wordList[pos:pos+5]):
                sakin=sakin+correction(kelime)+' '
        elif pos<=2 and len(wordList)-pos<=5:
            for i, kelime in enumerate(wordList[pos:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)-pos<1 and pos>3:
            for i, kelime in enumerate(wordList[pos-3:len(wordList)]):
                sakin=sakin+correction(kelime)+' '
        elif len(wordList)<3:
            for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        else:
             for i, kelime in enumerate(wordList):
                sakin=sakin+correction(kelime)+' '
        results = morphology.analyze(word)
        lemma=[]
        form=[]
        l=[]
        m=[]
        for i, result in enumerate(results):
            form.append(str(result.formatLong()))
            lemma.append(result.getLemmas()[0])
        if len(lemma)>1:
                analysis = morphology.analyzeSentence(sakin)
                results = morphology.disambiguate(sakin, analysis).bestAnalysis()
                for i, result in enumerate(results):
                        l.append(result.getLemmas()[0])
                        m.append(result.formatLong())
                for i in range(len(m)):
                    for j in range(len(form)):
                        if m[i]==form[j]:
                            lema=lemma[j]
                            if lema=='değil':
                                return 'değil'
                            if 'Neg' in form[j] or 'WithoutHavingDoneSo' in form[j] or 'Unable' in form[j]:
                                if checkOpennes(word):
                                    return lema+'me'
                                else:
                                    return lema+'ma'
                            if 'Without' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'siz'
                                elif PresentCheck(word)=='aı':
                                    return lema+'sız'
                                elif PresentCheck(word)=='uo':
                                    return lema+'suz'
                                else:
                                    return lema+'süz'
                            if 'With' in form[j]:
                                if PresentCheck(word)=='ei':
                                    return lema+'li'
                                elif PresentCheck(word)=='aı':
                                    return lema+'lı'
                                elif PresentCheck(word)=='uo':
                                    return lema+'lu'
                                else:
                                    return lema+'lü'
                            else:
                                return lema
                    else:
                        continue
        elif len(lemma)==1:
            if lemma[0]=='değil':
                return lemma[0]
            if 'Neg' in form[0] or 'WithoutHavingDoneSo' in form[0] or 'Unable' in form[0]:
                 if checkOpennes(word):
                    return lemma[0]+'me'
                 else:
                    return lemma[0]+'ma'
            elif 'Without' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'siz'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'sız'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'suz'
                else:
                    return lemma[0]+'süz'
            elif 'With' in form[0]:
                if PresentCheck(word)=='ei':
                    return lemma[0]+'li'
                elif PresentCheck(word)=='aı':
                    return lemma[0]+'lı'
                elif PresentCheck(word)=='uo':
                    return lemma[0]+'lu'
                else:
                    return lemma[0]+'lü'
            else:
                return lemma[0]
        else:
            return word




In [ ]:
def clean(text):
    text_list=text.split()
    for i, word in enumerate(text_list):
        text_list[i] = lemmatizer(word,text)
    return ' '.join(''.join(elems) for elems in text_list)

In [ ]:
comb["clean_new"]=comb["preProcessing"].map(clean) 

# 2nd round stopwords ve sil_1

In [ ]:
comb["clean_new"]=comb["clean_new"].map(sil_1)

In [ ]:
comb["clean_new"]=comb["clean_new"].map(stopwords_sil)

# Save to CSV

In [ ]:
comb.to_csv(r'drive/My Drive/(folder name)/(document name).csv', index=False) 